<a href="https://colab.research.google.com/github/Daniel-ASG/Web_Scraping/blob/main/deputados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Obter os dados dentro de toda a página

In [ ]:
import requests
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as  pd
from urllib.request import urlretrieve
import os

def trata_html(input):
    return ' '.join(input.split()).replace('> <', '><')

response = urlopen('https://www.camara.leg.br/deputados/quem-sao/resultado?search=&partido=&uf=&legislatura=56&sexo=')
html = response.read().decode('utf-8')
html = trata_html(html)
soup = BeautifulSoup(html, 'html.parser')

total_de_deputados = int(soup.find('div', {'class': 'busca-info__resultado busca-info__resultado--informado'}).getText().split(' ')[-2])
pages = int(total_de_deputados/25) + 1

url = []
cards = []


i=0
for page in range(pages):
    response = urlopen('https://www.camara.leg.br/deputados/quem-sao/resultado?search=&partido=&uf=&legislatura=56&sexo=&pagina=' + str(page+1))
    html = response.read().decode('utf-8')
    html = trata_html(html)
    soup = BeautifulSoup(html, 'html.parser')

    paginas = soup.findAll('li', {'class': 'lista-resultados__item'})

    for pagina in paginas:
        url.append('https://www.camara.leg.br' + pagina.a.get('href'))

        response =  urlopen(url[i])
        html = response.read().decode('utf-8')
        html = trata_html(html)
        ficha = BeautifulSoup(html, 'html.parser')

        response =  urlopen(url[i]+'/biografia')
        html = response.read().decode('utf-8')
        html = trata_html(html)
        bio = BeautifulSoup(html, 'html.parser')


        # cria novo card para cada deputado
        card = {}

        infos_bio = bio.find('ul', {'class': 'informacoes-deputado'}).findAll('li')
        n=0
        for info in infos_bio:
            if n>2:
                card[info.getText().split(':')[0]] = info.getText().split(':')[1].strip()
            n+=1

        infos = ficha.find('ul', {'class': 'informacoes-deputado'}).findAll('li')
        for info in infos:
            card[info.getText().split(':')[0]] = info.getText().split(':')[1].strip()

        card['Nome'] = ficha.find('h2', {'class': 'nome-deputado'}).getText()
        card['Partido'] = ficha.find('span', {'class': 'foto-deputado__partido-estado'}).getText().split(' - ')[0]
        card['Estado'] = ficha.find('span', {'class': 'foto-deputado__partido-estado'}).getText().split(' - ')[1]

        if ficha.find('title').getText().strip().split(' ')[0] == 'Deputada':
            card['Sexo'] = 'Feminino'
        else:
            card['Sexo'] = 'Masculino'
        
        # acrescenta o card deste deputado à lista cards
        cards.append(card)


        # dir = '/content/img/'
        # if not os.path.exists(dir):
        #     os.makedirs(dir)
        # urlretrieve(soup.img.get('src'), '/content/img/' + '_'.join(card['Nome'].split(' ')) + '.jpg')
        
        i+=1

dataset = pd.DataFrame(cards)
dataset[['Nome', 'Nome Civil', 'Partido', 'Estado', 'Naturalidade', 'Sexo', 'Data de Nascimento', 'Profissões', 'Escolaridade', 'E-mail', 'Telefone', 'Endereço']]